## Part 1

In [1]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2024, day=9)

def test(method, input, expected):
    actual = method(input)
    if actual == expected:
        print(f'\t☑ - {method.__name__}({input}) = {expected} = {actual}')
    else:
        print(f'\t☐ - {method.__name__}({input}) = {expected} ≠ {actual}')

In [2]:
import itertools

def parse_input(input): 
    files = input[::2]
    free_spaces = input[1::2]

    output = []

    for id, (file, free_space) in enumerate(list(itertools.zip_longest(files, free_spaces, fillvalue='0'))):
        output += str(id) * int(file)
        output += '.' * int(free_space)

    return ''.join(output)

In [3]:
test(parse_input, '12345', '0..111....22222')
test(parse_input, '90909', '000000000111111111222222222')
test(parse_input, '2333133121414131402', '00...111...2...333.44.5555.6666.777.888899')

	☑ - parse_input(12345) = 0..111....22222 = 0..111....22222
	☑ - parse_input(90909) = 000000000111111111222222222 = 000000000111111111222222222
	☑ - parse_input(2333133121414131402) = 00...111...2...333.44.5555.6666.777.888899 = 00...111...2...333.44.5555.6666.777.888899


In [4]:
parsed_input = list(parse_input(puzzle.examples[0].input_data))

for indx, value in enumerate(reversed(parsed_input.copy())):
    orignial_indx = len(parsed_input) - 1 - indx
    first_free_space = parsed_input.index('.')
    if not orignial_indx > first_free_space:
        break
    parsed_input[first_free_space] = value
    parsed_input[orignial_indx] = '.'

sum([int(x) * indx for (indx, x) in enumerate(parsed_input) if x != '.'])

1928

In [5]:
parsed_input = list(parse_input(puzzle.input_data))

for indx, value in enumerate(reversed(parsed_input.copy())):
    orignial_indx = len(parsed_input) - 1 - indx
    first_free_space = parsed_input.index('.')
    if not orignial_indx > first_free_space:
        break
    parsed_input[first_free_space] = value
    parsed_input[orignial_indx] = '.'

sum([int(x) * indx for (indx, x) in enumerate(parsed_input) if x != '.'])

90489586600

`90489586600` is too low.


I think what happened is that the ID is growing larger than 10 digits and getting mixed up. My parser is going to need to not return a string. It needs to return the array.

In [6]:
test(parse_input, '233313312141413140202', '00...111...2...333.44.5555.6666.777.8888991010')

	☑ - parse_input(233313312141413140202) = 00...111...2...333.44.5555.6666.777.8888991010 = 00...111...2...333.44.5555.6666.777.8888991010


In [7]:
import itertools

def parse_input(input): 
    files = input[::2]
    free_spaces = input[1::2]

    output = []

    for id, (file, free_space) in enumerate(list(itertools.zip_longest(files, free_spaces, fillvalue='0'))):
        output += [id] * int(file)
        output += ['.'] * int(free_space)

    return output

parsed_input = parse_input(puzzle.input_data)

for indx, value in enumerate(reversed(parsed_input.copy())):
    orignial_indx = len(parsed_input) - 1 - indx
    first_free_space = parsed_input.index('.')
    if not orignial_indx > first_free_space:
        break
    parsed_input[first_free_space] = value
    parsed_input[orignial_indx] = '.'

sum([int(x) * indx for (indx, x) in enumerate(parsed_input) if x != '.'])

6332189866718

## Part 2

This sounds like a sliding window algorithm or one of those text search algorithms. As always, I'm going to try brute force first :) I will need to adjust the structure of the parse if I'm going to get that going.


In [8]:
import itertools
from dataclasses import dataclass

@dataclass
class Block:
    id: int
    files: int
    free_space: int

def parse_input(input): 
    files = input[::2]
    free_spaces = input[1::2]

    output = []

    for id, (file, free_space) in enumerate(list(itertools.zip_longest(files, free_spaces, fillvalue='0'))):
        output.append(Block(id, int(file), int(free_space)))

    return output

blocks = parse_input(puzzle.examples[0].input_data)


for indx, block in enumerate(reversed(blocks.copy())):
    print(''.join([str(fs_part) for b in blocks for fs_part in ([b.id]*b.files + ['.']*b.free_space)]))
    print(blocks)
    print('\n')

    match = next((b for b in blocks if b.free_space >= block.files), None)
    
    if not match:
        print(f'{block.id} didn"t move')
        continue
    
    match_indx = blocks.index(match)

    if match_indx >= blocks.index(block):
        continue

    print(f'Moving {block.id} down next to {match.id}')

    blocks[blocks.index(block) - 1].free_space += block.free_space + block.files
    block.free_space = match.free_space - block.files
    match.free_space = 0

    blocks.remove(block)
    blocks.insert(match_indx + 1, block)

filesystem = [str(fs_part) for b in blocks for fs_part in ([b.id]*b.files + ['.']*b.free_space)]

print(''.join(filesystem))

sum([int(x) * indx for (indx, x) in enumerate(filesystem) if x != '.'])

00...111...2...333.44.5555.6666.777.888899
[Block(id=0, files=2, free_space=3), Block(id=1, files=3, free_space=3), Block(id=2, files=1, free_space=3), Block(id=3, files=3, free_space=1), Block(id=4, files=2, free_space=1), Block(id=5, files=4, free_space=1), Block(id=6, files=4, free_space=1), Block(id=7, files=3, free_space=1), Block(id=8, files=4, free_space=0), Block(id=9, files=2, free_space=0)]


Moving 9 down next to 0
0099.111...2...333.44.5555.6666.777.8888..
[Block(id=0, files=2, free_space=0), Block(id=9, files=2, free_space=1), Block(id=1, files=3, free_space=3), Block(id=2, files=1, free_space=3), Block(id=3, files=3, free_space=1), Block(id=4, files=2, free_space=1), Block(id=5, files=4, free_space=1), Block(id=6, files=4, free_space=1), Block(id=7, files=3, free_space=1), Block(id=8, files=4, free_space=2)]


8 didn"t move
0099.111...2...333.44.5555.6666.777.8888..
[Block(id=0, files=2, free_space=0), Block(id=9, files=2, free_space=1), Block(id=1, files=3, free_space=3)

2858

In [9]:
import itertools
from dataclasses import dataclass

@dataclass
class Block:
    id: int
    files: int
    free_space: int

def parse_input(input): 
    files = input[::2]
    free_spaces = input[1::2]

    output = []

    for id, (file, free_space) in enumerate(list(itertools.zip_longest(files, free_spaces, fillvalue='0'))):
        output.append(Block(id, int(file), int(free_space)))

    return output

blocks = parse_input(puzzle.input_data)


for indx, block in enumerate(reversed(blocks.copy())):
    # print(''.join([str(fs_part) for b in blocks for fs_part in ([b.id]*b.files + ['.']*b.free_space)]))
    # print(blocks)
    # print('\n')

    match = next((b for b in blocks if b.free_space >= block.files), None)
    
    if not match:
        print(f'{block.id} didn"t move')
        continue
    
    match_indx = blocks.index(match)

    if match_indx >= blocks.index(block):
        continue

    # print(f'Moving {block.id} down next to {match.id}')

    blocks[blocks.index(block) - 1].free_space += block.free_space + block.files
    block.free_space = match.free_space - block.files
    match.free_space = 0

    blocks.remove(block)
    blocks.insert(match_indx + 1, block)

filesystem = [str(fs_part) for b in blocks for fs_part in ([b.id]*b.files + ['.']*b.free_space)]

# print(''.join(filesystem))

sum([int(x) * indx for (indx, x) in enumerate(filesystem) if x != '.'])

6353648390778